In [1]:
import os
import pandas as pd
from openai import OpenAI
from pinecone import Pinecone
from dotenv import load_dotenv

load_dotenv()

openai_api_key = os.getenv("OPENAI_API_KEY")
pinecone_api_key = os.getenv("PINECONE_API_KEY")




In [ ]:
#  OpenAI 클라이언트
client = OpenAI(api_key=openai_api_key)

def get_embedding(text, model="text-embedding-3-small"):
    response = client.embeddings.create(model=model, input=text)
    return response.data[0].embedding

In [3]:
pc = Pinecone(api_key=pinecone_api_key)

In [ ]:
#  인덱스 연결
index = pc.Index("fooddata")  # 콘솔에서 만든 인덱스 이름과 동일해야 함

c:\Users\Admin\miniconda3\envs\prompting_env\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
# 6. CSV 불러오기
df = pd.read_csv("foods_clean.csv")

# content 컬럼 생성 (모든 컬럼 결합)
def make_text(row):
    return ", ".join([f"{col}: {row[col]}" for col in df.columns])

df["content"] = df.apply(make_text, axis=1)

In [10]:
batch_size = 100
vectors = []

for i in range(0, len(df), batch_size):
    batch_texts = df["content"].iloc[i:i+batch_size].tolist()

    # OpenAI 배치 임베딩 요청
    response = client.embeddings.create(
        model="text-embedding-3-small",
        input=batch_texts
    )
    embeddings = [d.embedding for d in response.data]

    # Pinecone 업서트용 벡터 생성
    batch_vectors = []
    for j, (text, emb) in enumerate(zip(batch_texts, embeddings)):
        idx = i + j  # 실제 행 번호
        metadata = df.iloc[idx].to_dict()
        batch_vectors.append({
            "id": f"food_{idx}",
            "values": emb,
            "metadata": metadata
        })

    # Pinecone 업서트
    index.upsert(vectors=batch_vectors, namespace="foods-ns1")

    print(f"{i+len(batch_vectors)}/{len(df)} 업서트 완료")


100/69844 업서트 완료
200/69844 업서트 완료
300/69844 업서트 완료
400/69844 업서트 완료
500/69844 업서트 완료
600/69844 업서트 완료
700/69844 업서트 완료
800/69844 업서트 완료
900/69844 업서트 완료
1000/69844 업서트 완료
1100/69844 업서트 완료
1200/69844 업서트 완료
1300/69844 업서트 완료
1400/69844 업서트 완료
1500/69844 업서트 완료
1600/69844 업서트 완료
1700/69844 업서트 완료
1800/69844 업서트 완료
1900/69844 업서트 완료
2000/69844 업서트 완료
2100/69844 업서트 완료
2200/69844 업서트 완료
2300/69844 업서트 완료
2400/69844 업서트 완료
2500/69844 업서트 완료
2600/69844 업서트 완료
2700/69844 업서트 완료
2800/69844 업서트 완료
2900/69844 업서트 완료
3000/69844 업서트 완료
3100/69844 업서트 완료
3200/69844 업서트 완료
3300/69844 업서트 완료
3400/69844 업서트 완료
3500/69844 업서트 완료
3600/69844 업서트 완료
3700/69844 업서트 완료
3800/69844 업서트 완료
3900/69844 업서트 완료
4000/69844 업서트 완료
4100/69844 업서트 완료
4200/69844 업서트 완료
4300/69844 업서트 완료
4400/69844 업서트 완료
4500/69844 업서트 완료
4600/69844 업서트 완료
4700/69844 업서트 완료
4800/69844 업서트 완료
4900/69844 업서트 완료
5000/69844 업서트 완료
5100/69844 업서트 완료
5200/69844 업서트 완료
5300/69844 업서트 완료
5400/69844 업서트 완료
5500/69844 업서트 완료
5600/69844 업서트 완료
5

In [12]:
from openai import OpenAI

client = OpenAI(api_key=openai_api_key)

def query_food_with_reasoning(prompt, top_k=5):
    # 1) 사용자 질문 임베딩
    query_embedding = client.embeddings.create(
        model="text-embedding-3-small",
        input=prompt
    ).data[0].embedding

    # 2) Pinecone 검색
    results = index.query(
        namespace="foods-ns1",
        vector=query_embedding,
        top_k=top_k,
        include_metadata=True
    )

    # 3) 검색된 음식 모으기
    candidates = []
    for match in results["matches"]:
        food_name = match["metadata"].get("food_name", "Unknown")
        kcal = match["metadata"].get("energy_kcal", "N/A")
        protein = match["metadata"].get("protein_g", "N/A")
        candidates.append(f"{food_name} ({kcal}kcal, 단백질 {protein}g)")

    # 4) GPT에게 추천 문장 생성시키기
    system_prompt = """너는 개인 맞춤 식단 관리 전문가야.
    사용자가 먹은 음식과 Pinecone에서 검색된 음식 후보를 참고해서
    저녁에 어떤 음식을 먹으면 좋은지 추천해줘.
    """

    user_prompt = f"사용자 질문: {prompt}\n\n추천 후보 음식: {', '.join(candidates)}"

    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt}
        ]
    )

    return response.choices[0].message.content

# 실행 예시
answer = query_food_with_reasoning("나는 44살 여성이야. 오늘 점심에 햄버거와 콜라를 먹었는데 저녁은 어떻게 먹어야 좋을까", top_k=5)
print(answer)


점심으로 햄버거와 콜라를 드셨으니, 저녁은 좀 더 가벼운 음식을 선택하는 것이 좋을 것 같습니다. 이미 오늘 높은 칼로리의 음식을 섭취하셨기 때문에, 저녁은 영양소는 충분하면서도 칼로리는 낮은 옵션으로 추천드릴게요.

추천하는 저녁 메뉴는:

- **채소 샐러드**: 신선한 채소와 드레싱으로 가벼운 식사를 하세요. 단백질원을 추가하고 싶다면 grilled chicken이나 병아리콩을 넣어도 좋습니다.

또는 자세한 음식 후보 중에서 선택 하신다면, 추천할 수 있는 옵션이 여러 개 있지만, 만약 아래의 햄버거를 고려한다면 칼로리를 체크하시고 섭취량을 조절하는 것이 좋습니다:

- **햄버거 (229.0kcal, 단백질 8.0g)**: 저녁에는 한 개 정도로 제한하는 것이 좋을 것 같습니다. 하지만 다른 더 건강한 옵션과 함께 고민해보세요.

간단하게 과일이나 요거트를 곁들이면 좋은 저녁 식사가 될 것입니다. 충분한 수분 섭취도 잊지 마세요!


In [13]:
answer = query_food_with_reasoning(
    "나는 44세 여성이고 현재 60kg인데 55kg까지 빼고 싶어. "
    "하루 권장 칼로리는 2000kcal 정도인데, "
    "일주일 동안 균형 잡힌 식단을 짜줘. "
    "조건: 하루 섭취 1500~1700kcal, 아침/점심/저녁 3끼로 표로 정리, "
    "단백질 충분히, 과일/채소 포함, 군것질/야식 제외.",
    top_k=10
)
print(answer)

아래는 7일간의 균형 잡힌 식단입니다. 이 식단은 매일 1500~1700kcal를 바탕으로 하며, 단백질이 풍부하고 과일과 채소를 포함하고 있습니다. 

| 요일  | 아침                                               | 점심                                               | 저녁                                               | 총 칼로리 |
|-------|------------------------------------------------|------------------------------------------------|------------------------------------------------|----------|
| 월요일 | 하루근력 (70.0kcal, 단백질 3.3g) + 과일 1종 (100kcal) | 칼집 생왕구이 (144.0kcal, 단백질 14.0g) + 샐러드 (120kcal) | 지친몸을달래고싶은날엔해초전복죽 (55.0kcal, 단백질 2.42g) + 채소 (50kcal) | 535.0kcal + 50kcal + 120kcal + 100kcal = 805kcal |
| 화요일 | 하루쿡 깍두기 (38.0kcal, 단백질 2.4g) + 요거트 (100kcal) | 그남자그여자 근육찾기 카페라떼맛 (393.0kcal, 단백질 66.0g) + 채소 (50kcal) | 하루근력 (70.0kcal, 단백질 3.3g) + 과일 1종 (100kcal) | 651.0kcal + 50kcal + 100kcal = 801kcal |
| 수요일 | 칼로리반견과곤약죽 (75.0kcal, 단백질 2.38g) + 과일 1종 (100kcal) | 칼집 생왕구이 (144.0kcal, 단백질 14.0g) + 쌈채소 (50kcal) | 너의하루 쇼콜라 (410.0kcal, 단백질 26.0g) + 샐러드 (120kcal) | 410.0kcal +

In [14]:
answer = query_food_with_reasoning(
    "나는 44세 여성이고 현재 60kg인데 55kg까지 감량하고 싶어. "
    "어제 저녁에 삼겹살과 소주 2병을 마셔서 칼로리를 초과했어. "
    "이걸 보정할 수 있도록 일주일 치 균형 잡힌 식단을 다시 짜줘. "
    "조건: 하루 1500~1700kcal, 아침/점심/저녁 3끼 표 형식, "
    "단백질 충분히, 과일/채소 포함, 군것질·야식·술 제외, "
    "회식으로 인한 초과분을 어떻게 보정했는지도 설명해줘.",
    top_k=10
)
print(answer)

당신의 목표에 맞춰 1주일 간의 균형 잡힌 식단을 제안해드리겠습니다. 하루 1500~1700kcal의 범위 내에서 단백질이 충분하도록 하며, 과일과 채소도 포함하도록 하겠습니다. 또한, 어제 저녁의 칼로리 초과를 보정할 수 있는 방법도 제안하겠습니다.

### 1주일 식단 계획

| 요일  | 아침                         | 점심                             | 저녁                             | 총칼로리(kcal)  |
|-------|------------------------------|----------------------------------|----------------------------------|------------------|
| 월요일| 칼로리반닭가슴살곤약죽 (38) | 샐러드 (닭가슴살, 야채) (350)   | 내몸을 위한 최담 (42)           | 430              |
| 화요일| 진가참전복죽 (50)           | 통곡물밥 + 두부조림 + 채소 (500)| 지친몸을달래고싶은날엔해초전복죽 (55)| 605              |
| 수요일| 하루한죽비트닭가슴살죽 (60) | 퀴노아 샐러드 (300)           | 삼겹살볶음_세발낙지 (225)      | 585              |
| 목요일| 칼로리반닭가슴살곤약죽 (38) | 샐러드 + 아보카도 (400)       | 삼겹살볶음_낙지 (235)          | 673              |
| 금요일| 진가참전복죽 (50)           | 현미밥 + 나물무침 (500)        | 내몸을 위한 최담 (42)           | 592              |
| 토요일| 하루한죽비트닭가슴살죽 (60) | 닭가슴살 + 야채 볶음 (350)     | 지친몸을달래고싶은날엔해초전복죽 (55)| 465              |
| 일요일| 칼로리반닭가슴살

In [15]:
answer = query_food_with_reasoning(
    "나는 30대 남성이며 현재 체중은 64kg이고, "
    "주 2회 헬스장에서 웨이트 운동을 한다. "
    "근육을 키우고 체중을 68kg까지 증량하고 싶다. "
    "하루 총 칼로리를 2300~2600kcal 정도로 맞추고, "
    "단백질은 체중 1kg당 1.6~2g 이상 포함되게 해줘. "
    "일주일치 식단을 표 형식으로 작성하되, "
    "일주일에 한 번 회식(삼겹살 2인분 + 소주 1병 + 맥주 1병)이 있다고 가정하고 "
    "그 날은 나머지 끼니에서 칼로리와 영양을 어떻게 조절할지도 함께 제시해줘.",
    top_k=10
)
print(answer)

안녕하세요! 근육을 키우고 체중을 증량하기 위한 일주일치 식단을 제안드리겠습니다. 총 열량과 단백질 섭취를 고려하여 설계하였고, 주 한 번의 회식 날에도 조절할 수 있는 방법을 포함했습니다.

### 일주일 식단 계획

| 요일 | 아침 (kcal)                         | 점심 (kcal)                          | 저녁 (kcal)                                 | 간식 (kcal)                           | 총 칼로리 (kcal) | 단백질 (g) |
|------|-------------------------------------|--------------------------------------|----------------------------------------------|---------------------------------------|------------------|--------------|
| 월요일 | 닭가슴살 100g + 고구마 150g (300) | 현미밥 200g + 연어구이 (500)        | 낭만포차 꼬치어묵 우동 2세트 (278)        | 그남자그여자 근육찾기 카페라떼맛 (393) |  1971          |  77       |
| 화요일 | 오트밀 50g + 우유 (300)           | 쇠고기 불고기 150g + 야채 (600)    | 난자완스 (173)                             | 소견과 오렌지 한 개 (120)             |  2193          |  65       |
| 수요일 | 스크램블 에그 3개 + 토스트 (400) | 닭가슴살 샐러드 (500)              | 헤자로운 매콤볶음우동정식 (171)            | 견과류 30g (182)                       |  2233          |  55    